In [ ]:
#default_exp data.load

In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.notebook.showdoc import show_doc

In [ ]:
#export
from torch.utils.data.dataloader import _MultiProcessingDataLoaderIter,_SingleProcessDataLoaderIter,_DatasetKind
_loaders = (_MultiProcessingDataLoaderIter,_SingleProcessDataLoaderIter)

## DataLoader

In [ ]:
class SleepyDS():
    def __init__(self,coll): self.coll,self.rng = coll,random.Random()
    def __len__(self): return len(self.coll)
    def __getitem__(self,i):
        time.sleep(self.rng.random()/100)
        return self.coll[i]

def twoepochs(d): return ' '.join(''.join(o) for _ in range(2) for o in d)

testds = SleepyDS(string.ascii_lowercase)    
bs = 4

In [ ]:
# class _NextFetcher:
#     def __init__(self, dataset): self.dataset_iter = iter(dataset)
#     def fetch(self, possibly_batched_index): return next(self.dataset_iter)
# def create_fetcher(kind, dataset, auto_collation, collate_fn, drop_last): return _NextFetcher(dataset)
# _DatasetKind.create_fetcher = create_fetcher

- set bs,drop_last,sampler after init
- collate_fn,kind,sampler,auto_collate from ds
- figure ds type from attr, not inheritance
- transforms
- reset()

In [ ]:
def delegate_attr(k, o, to):
    if k.startswith('_') or k==to: raise AttributeError(k)
    try: return getattr(getattr(o,to), k)
    except AttributeError: raise AttributeError(k) from None

In [ ]:
#export
def _wif(worker_id):
    info = get_worker_info()
    ds = info.dataset
    ds.nw,ds.offs = info.num_workers,info.id
    ds.wif()

In [ ]:
#     def _delegate_items(self, *attrs):
#         for attr in attrs:
#             if hasattr(self.items,attr): setattr(self, attr, getattr(self.items, attr))

In [ ]:
#export
class Dataset():
    @kwargs('collate_fn indexes batches reset wif')
    def __init__(self, items=None, **kwargs):
        if items is not None and hasattr(items,'__dict__'): items.wrapper = self
        self.items = items
        self.rng,self.sampler = random.Random(),itertools.count()
        for k,v in kwargs.items(): setattr(self,k,types.MethodType(v,self))

    def __iter__(self):
        torch.manual_seed(self.rng.randint(0,sys.maxsize))
        self.reset()
        for b in self.batches(self.indexes()):
            yield b if isinstance(b,Tensor) else self.collate_fn(b)

    @classmethod
    def create(cls, items, **kwargs): return IndexedDataset(items, **kwargs)
    
    def __getattr__(self,k): return delegate_attr(k,self,'items')
    def collate_fn(self, b): return default_collate(b)
    def indexes(self): return None
    def batches(self, idxs): return self.items
    def __len__(self): return len(self.sampler)
    def reset(self): pass
    def wif(self): pass

In [ ]:
ds1 = Dataset(testds)
test_eq(''.join(ds1), string.ascii_lowercase)

t2 = [tensor([0,1,2]),tensor([3,4,5])]
ds2 = Dataset(t2)
test_eq(list(ds2), t2)

t3 = [[0,1,2],[3,4,5]]
ds3 = Dataset(t3)
test_eq_type(list(ds3), t2)

ds4 = Dataset(t3, collate_fn=noops)
test_eq(list(ds4), t3)

In [ ]:
#export
class DataLoader:
    _auto_collation,collate_fn,drop_last,dataset_kind = False,noops,False,_DatasetKind.Iterable
    def __init__(self, dataset, num_workers=0, pin_memory=False, timeout=0, tfm=noop, **kwargs):
        self.dataset = dataset if isinstance(dataset, Dataset) else Dataset.create(dataset, **kwargs) 
        self.pin_memory,self.tfm,self.worker_init_fn = pin_memory,tfm,_wif
        self._index_sampler = self.dataset.sampler
        self.num_workers = 0 if num_workers < 0 else num_workers
        self.timeout = 0 if timeout < 0 else timeout

    def __iter__(self):  return map(self.tfm, _loaders[self.num_workers==0](self))
    def __getattr__(self,k): return delegate_attr(k,self,'dataset')
    def __len__(self): return len(self.dataset)

In [ ]:
test_eq(''.join(DataLoader(ds1, num_workers=0)), string.ascii_lowercase)
test_eq(list(DataLoader(ds2, num_workers=1)), t2)
# n workers means n copies of the iter, in some arbitrary order
test_eq(list(sorted(DataLoader(ds4, num_workers=2))), list(sorted(t3*2)))

In [ ]:
def set_sampler(dataset):
    if not batch_size or batch_sampler: assert not shuffle and sampler and not drop_last
    if batch_sampler: return batch_sampler
    if sampler: assert not shuffle, "Can't shuffle a custom sampler"
    else: sampler = (_InfiniteConstantSampler() if dataset.is_iterable
                     else (SequentialSampler,RandomSampler)[shuffle](dataset))
    return BatchSampler(sampler, batch_size, drop_last)

def set_collate_fn(self):
    self.collate_fn = return _utils.collate.default_collate if self._auto_collation else _utils.collate.default_convert)

In [ ]:
#export
class IndexedDataset(Dataset):
    def __init__(self, items ,bs=1, shuffle=False, sampler=None, batch_sampler=None, drop_last=False,
                 sampler_cls=None, batch_sampler_cls=BatchSampler, collate_fn=default_collate):
        super().__init__(items,collate_fn)
        self.sampler = batch_sampler
        self.rng,self.nw,self.offs = random.Random(),1,0
        self._delegate_items("get_batches","get_batch","collate")
        if self.sampler: return
        if not sampler: sampler = ifnone(sampler_cls, (SequentialSampler,RandomSampler)[shuffle])(items)
        self.sampler = batch_sampler_cls(sampler, bs, drop_last)

    def __iter__(self):
        torch.manual_seed(self.rng.randint(0,sys.maxsize))
        samps = list(enumerate(self.sampler))
        idxs = (b for i,b in samps if i%self.nw==self.offs)
        return self.get_batches(idxs)
    
    def get_batch(self, b): return [self.items[j] for j in b]
    def get_batches(self, idxs): return map(self.get_batch, idxs)
    def wif(self) : self.sampler.sampler = copy(self.sampler.sampler)

In [ ]:
def get_dl(bs=1, collate_fn=default_collate, num_workers=0, **kwargs):
    return DataLoader(testds, num_workers=num_workers, bs=bs, collate_fn=collate_fn, **kwargs)

In [ ]:
dl = get_dl(bs=4, num_workers=0)
t = twoepochs(dl)
test_eq(t, 'abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz')
test_eq(len(set(t)), 27)

In [ ]:
dl = get_dl(bs=4, num_workers=4, shuffle=True)
t = twoepochs(dl)
test_ne(t, 'abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz')
test_eq(len(set(t)), 27)
t

'vaqd nyfi meob jgps hwtu rzkc lx vaqd nyfi meob jgps hwtu rzkc lx'

## BatchDS

In [ ]:
#export
class BaseDS(GetAttr):
    _xtra = ['show', 'decode', 'show_at', 'decode_at', 'decode_batch']
    def __init__(self, ds):
        self.default = self.ds = ds
        ds.wrapper = self
        self._delegate_ds("reset")

    def _delegate_ds(self, attr):
        if hasattr(self.ds,attr): setattr(self, attr, getattr(self.ds, attr))
            
    def reset(self): pass

In [ ]:
#export
class BatchDS(BaseDS, IterableDataset):
    _xtra = ['show', 'decode', 'show_at', 'decode_at', 'decode_batch']
    def __init__(self, ds ,bs=1, shuffle=False, sampler=None, batch_sampler=None, drop_last=False,
                 collate_fn=default_collate, sampler_cls=None, batch_sampler_cls=BatchSampler):
        self.default,self.ds,self.samp,self.collate_fn = ds,ds,batch_sampler,collate_fn
        self.rng,self.nw,self.offs,self.is_iterable = random.Random(),1,0,True
        for o in ("get_batches","get_batch","collate"): self._delegate_ds(o)
        if self.samp: return
        if not sampler: sampler = ifnone(sampler_cls, (SequentialSampler,RandomSampler)[shuffle])(ds)
        self.samp = batch_sampler_cls(sampler, bs, drop_last)

    def __iter__(self):
        torch.manual_seed(self.rng.randint(0,sys.maxsize))
        samps = list(enumerate(self.samp))
        idxs = (b for i,b in samps if i%self.nw==self.offs)
        return self.get_batches(idxs)
    
    def get_batch(self, b): return [self.ds[j] for j in b]
    def get_batches(self, idxs): return map(self.get_batch, idxs)
    def collate(self, idxs): return self.collate_fn(self.get_batches(idxs))
    def __len__(self): return len(self.samp)

In [ ]:
#export
def _wif(worker_id):
    info = get_worker_info()
    ds = info.dataset
    ds.nw,ds.offs = info.num_workers,info.id
    ds.samp.sampler = copy(ds.samp.sampler)

In [ ]:
class SleepyDS():
    def __init__(self,coll): self.coll=coll
    def __len__(self): return len(self.coll)
    def __getitem__(self,i): time.sleep(0.02); return self.coll[i]

In [ ]:
letters = list(string.ascii_lowercase)
def twoepochs(d): print(' '.join(''.join(o) for _ in range(2) for o in d))
bs = 4

In [ ]:
#export
def dataloader(ds, bs=1, num_workers=0, collate_fn=default_collate, **kwargs):
    if not isinstance(ds, IterableDataset): ds = BatchDS(ds, bs, **kwargs)
    return DataLoader(ds, num_workers=num_workers, batch_size=None,
                      worker_init_fn=_wif, collate_fn=noop)

In [ ]:
def get_dl(bs=1, collate_fn=default_collate, num_workers=0, **kwargs):
    return dataloader(SleepyDS(string.ascii_lowercase), bs, num_workers, collate_fn=collate_fn, **kwargs)

In [ ]:
dl = get_dl(bs=4, num_workers=4, shuffle=True)
twoepochs(dl)
test_eq(len(set(sum(dl,[]))), 26)

gdxl ubiy znwj ecra spkh mtfq vo gdxl ubiy znwj ecra spkh mtfq vo


In [ ]:
dl = get_dl(bs=4, num_workers=4)
%time twoepochs(dl)
test_eq(len(set(sum(dl,[]))), 26)

abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz
CPU times: user 21.1 ms, sys: 46.4 ms, total: 67.5 ms
Wall time: 396 ms


In [ ]:
dl = get_dl(bs=4, num_workers=0)
%time twoepochs(dl)
len(dl)

abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz
CPU times: user 5.71 ms, sys: 0 ns, total: 5.71 ms
Wall time: 1.05 s


7

In [ ]:
dl = get_dl(bs=4, num_workers=4, drop_last=True)
twoepochs(dl)
len(dl)

abcd efgh ijkl mnop qrst uvwx abcd efgh ijkl mnop qrst uvwx


6

In [ ]:
dl = get_dl(bs=4, num_workers=0, shuffle=True)
twoepochs(dl)
test_eq(len(set(sum(dl,[]))), 26)

lahp kzyn rgmb sfdt xvco ueij wq svid tckw phjz raeu gfqy mlnb xo


In [ ]:
ds = SleepyDS(string.ascii_lowercase)
dl = get_dl(bs=4, num_workers=4, sampler=SequentialSampler(ds))
twoepochs(dl)

abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz


In [ ]:
dl = get_dl(num_workers=4, batch_sampler=BatchSampler(RandomSampler(ds), 8, False))
twoepochs(dl)
test_eq(len(set(sum(dl,[]))), 26)

ypojsdrz hkvwnilc xqmbfgtu ae ypojsdrz hkvwnilc xqmbfgtu ae


In [ ]:
def rev_collate(s): return default_collate(list(reversed(s)))
dl = get_dl(bs=4, num_workers=4, collate_fn=rev_collate)
twoepochs(dl)

abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz


In [ ]:
class SleepyDS2(SleepyDS):
    def get_batch(self, b): return "".join([self[j] for j in b]) + '/'

dl = dataloader(SleepyDS2(string.ascii_lowercase), bs=4, num_workers=4)
twoepochs(dl)

abcd/ efgh/ ijkl/ mnop/ qrst/ uvwx/ yz/ abcd/ efgh/ ijkl/ mnop/ qrst/ uvwx/ yz/
